# Recurrent postprocessing: v1
This file takes as input a sequence of data from the CNN, and attempts to refine that into a more accurate output command.

Some credit belongs to https://github.com/harvitronix/five-video-classification-methods/blob/master/models.py for providing inspiration.

In [2]:
#Create references to important directories we will use over and over
import os, sys
DATA_HOME_DIR = '/home/nathan/olin/spring2017/line-follower/line-follower/data'

In [3]:
#import modules
import numpy as np
from glob import glob
from PIL import Image
from tqdm import tqdm
import bcolz

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
from keras.layers import *
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.wrappers import TimeDistributed
from keras.metrics import categorical_crossentropy, categorical_accuracy

Using TensorFlow backend.


In [5]:
%cd $DATA_HOME_DIR

path = DATA_HOME_DIR
train_path=path + '/qea-square_2'
valid_path=path + '/qea-square_3'

/home/nathan/olin/spring2017/line-follower/line-follower/data


## Gather data

In [6]:
INPUT_LEN = 512 # The number of columns in the CSV
WINDOW_SIZE = 10

In [7]:
def load_array(fname):
    return bcolz.open(fname)[:]

def concatCmdVelFlip(array):
    """ Concatentaes and returns Cmd Vel array """
    return np.concatenate((array, array*-1)) # multiply by negative 1 for opposite turn

def windows(X, Y, seq_len=10):
    assert len(X) == len(Y)
    
    result = []
    for index in range(X.shape[0] - seq_len):
        result.append([X[index:index+seq_len], Y[index+seq_len-1]])
    result = np.array(result)
    np.random.shuffle(result)
    return np.array(list(result[:,0])), np.array(list(result[:,1]))

def loadData(folder):
    Y = np.genfromtxt(folder+'/cmd_vel.csv', delimiter=',')[:,1] # only use turning angle
    Y = concatCmdVelFlip(Y)
    
#     X_all = np.genfromtxt(folder+'/X_train_preds.csv', delimiter=',')
    X_all = load_array(folder+'/X_train_features.b')
    X_all = np.reshape(X_all, (len(X_all), INPUT_LEN))
    
    X_windowed, Y_windowed = windows(X_all, Y, WINDOW_SIZE)
    
    return X_windowed, Y_windowed

In [8]:
X_train, Y_train = loadData(train_path);
X_valid, Y_valid = loadData(valid_path);

## Network

In [9]:
in_shape = (WINDOW_SIZE, INPUT_LEN)

In [10]:
def get_model():
    model = Sequential([
            LSTM(2048, return_sequences=False, input_shape=in_shape),
            Dropout(0.5),
#             Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(1)
        ])
    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    return model
    
model = get_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 2048)          20979712    lstm_input_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 2048)          0           lstm_1[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           1049088     dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 512)           0           dense_1[0][0]                    
___________________________________________________________________________________________

### Train the model

In [14]:
history = model.fit(X_train, Y_train,
                    batch_size = 96,
                    nb_epoch=150,
                    validation_data=(X_valid, Y_valid),
                    verbose=True)

Train on 508 samples, validate on 180 samples
Epoch 1/150
508/508 [==============================] - 0s - loss: 0.0859 - val_loss: 0.1005
Epoch 2/150
508/508 [==============================] - 0s - loss: 0.0805 - val_loss: 0.0961
Epoch 3/150
508/508 [==============================] - 0s - loss: 0.0880 - val_loss: 0.0960
Epoch 4/150
508/508 [==============================] - 0s - loss: 0.0845 - val_loss: 0.1035
Epoch 5/150
508/508 [==============================] - 0s - loss: 0.0859 - val_loss: 0.1049
Epoch 6/150
508/508 [==============================] - 0s - loss: 0.0835 - val_loss: 0.1152
Epoch 7/150
508/508 [==============================] - 0s - loss: 0.0816 - val_loss: 0.1164
Epoch 8/150
508/508 [==============================] - 0s - loss: 0.0824 - val_loss: 0.0907
Epoch 9/150
508/508 [==============================] - 0s - loss: 0.0886 - val_loss: 0.0991
Epoch 10/150
508/508 [==============================] - 0s - loss: 0.0849 - val_loss: 0.1009
Epoch 11/150
508/508 [===========

In [13]:
conv_predictions = X_valid[:,-1]
recurrent_predictions = model.predict(X_valid)
ground_truth = Y_valid
for x,y,z in zip(conv_predictions, ground_truth, recurrent_predictions):
    print ("{:07f}\t{:07f}\t".format(y,z[0]))

0.014183	0.003948	
0.000000	-0.031058	
-0.300000	0.009484	
-0.300000	0.005337	
-0.000000	-0.006013	
-0.009191	-0.015654	
0.046632	0.003594	
-0.000000	-0.004185	
0.000000	0.000471	
-0.119028	-0.005344	
0.213890	0.003605	
0.064105	0.017826	
-0.031636	-0.001545	
0.000000	-0.005993	
-0.300000	0.010110	
-0.014183	0.000652	
-0.104052	-0.132000	
-0.000000	-0.029337	
0.000000	-0.010760	
-0.126517	-0.004734	
0.000000	-0.024179	
-0.086550	-0.032931	
-0.213890	-0.006575	
-0.178935	0.018968	
0.163958	0.001379	
-0.000000	0.003433	
-0.213890	0.139114	
-0.029159	-0.000960	
-0.046632	-0.143306	
0.136501	0.014278	
-0.000000	0.001673	
-0.233858	-0.051530	
0.000000	0.001288	
-0.000000	0.002375	
0.263811	-0.036241	
0.099040	0.021021	
0.031636	-0.002330	
0.000000	-0.002195	
-0.000000	-0.025079	
-0.138997	-0.000329	
0.021652	-0.067185	
-0.300000	0.001078	
0.216386	-0.042051	
-0.300000	-0.025090	
-0.129013	0.001485	
-0.216386	0.114091	
0.300000	-0.043524	
0.300000	-0.005425	
0.000000	0.032213	
-0.049128	0.00

## Analyze training

In [ ]:
sns.tsplot(history.history['val_loss'])